In [1]:
import json, requests
import gspread
import datetime
from oauth2client.service_account import ServiceAccountCredentials

# Get date time
def get_time():
	now = str(datetime.datetime.now())[:10]
	return now
	
# Get bittrex change
def get_bittrex_change(yesterday_price, current_price):
	change = (current_price - yesterday_price)*100/yesterday_price
	return change 
	
# Get bittrex prices
def get_bittrex():

	# Get pair
	bittrex = {}
	bittrex["time"] = get_time()
	url1 = "https://bittrex.com/api/v1.1/public/getmarkets"
	resp = requests.get(url=url1)
	data = json.loads(resp.text)
	
	
	# Get pairs 
	btc_bittrex = []
	for i in data["result"]:
		if "BTC" in i["MarketName"]:
			btc_bittrex.append(i["MarketName"])
		else:
			pass
			
	# Get pairs values
	bittrex["pair"] = {}
	
	for name in btc_bittrex:
		url2 = "https://bittrex.com/api/v1.1/public/getmarketsummary?market=" + name
		resp2 = requests.get(url=url2)
		data2 = json.loads(resp2.text)
		sub_data2 = {}
		
		# Extract data from data2
		sub_data2["name"] 	= name
		sub_data2["last_price"] = data2["result"][0]["Last"]
		sub_data2["volume"] 	= data2["result"][0]["Volume"]
		sub_data2["change"] 	= get_bittrex_change(data2["result"][0]["PrevDay"], data2["result"][0]["Last"])
		# print(sub_data2)
		bittrex["pair"][name]  = sub_data2

	return bittrex
# bittrex = get_bittrex()

# Get binance prices
def get_binance():

    binance         = {}
    binance["time"] = get_time()
    binance["pair"] = {}
    #sub_data2      = {}
    url             = "https://www.binance.com/api/v1/ticker/24hr"
    resp            = requests.get(url=url)
    data            = json.loads(resp.text)

    # Get pairs
    btc_binance = []
    for item in data:
        if "BTC" in item["symbol"]:
		
	    # Extract data
            sub_data2               = {}
            sub_data2["name"]       = item["symbol"]
            sub_data2["last_price"] = float(item["lastPrice"])
            sub_data2["volume"]     = float(item["volume"])
            sub_data2["change"]     = float(item["priceChangePercent"])
            # print item["symbol"]
            binance["pair"][item["symbol"]] = sub_data2
        else:
            pass
    return binance
# binance = get_binance()

In [2]:
binance = get_binance()
bittrex = get_bittrex()

In [26]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [43]:
# Auth
spreadsheet_name = "Bittrex and Binance monthly data"
email            = "tomcangbk@gmail.com"
key              = "client_secret.json"

scope 		= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(key, scope)
gc 			= gspread.authorize(credentials)

In [44]:
# Open spreadsheet if exist. If not, create it!
try:
    sh          = gc.open(spreadsheet_name)
    sh.share(email, perm_type='user', role='writer')
    print "Success open spreadsheet"
except:
    print "No spreadsheet name " + spreadsheet_name + "create spreadsheet"
    sh = gc.create(spreadsheet_name)
    sh.share(email, perm_type='user', role='writer')

Success open spreadsheet


In [45]:
col_bittrex = len(bittrex["pair"])*3 + 1
col_binance = len(binance["pair"])*3 + 1

bittrex_sheet_name      = get_time()[:7] + "-bittrex"
print bittrex_sheet_name
binance_sheet_name      = get_time()[:7] + "-binance"
print binance_sheet_name
coin = {bittrex_sheet_name: {"data": bittrex, "column": col_bittrex}, 
        binance_sheet_name: {"data": binance, "column": col_binance}}

2018-01-bittrex
2018-01-binance


In [46]:
for k, v in coin.iteritems():
    try:
        ws = sh.worksheet(k)
        # Add new data to exist sheet
        print "da co sheet " + k
        
        # Finding "count"
        count = 0
        for i in range(2, 33):
            if ws.cell(1, i).value != "":
                pass
            else:
                count = i + 1
                print count
                break
        ws.update_cell(count, 1, v["data"]["time"])
        # Append data to exist sheet
        for key, value in v["data"]["pair"].iteritems():
            # Find pair if exist and write data
            try:
                cell = ws.find(key)
                i = cell.col
                ws.update_cell(count, i, value["last_price"])
                ws.update_cell(count, i+1, value["volume"])
                ws.update_cell(count, i+2, value["change"])
            except gspread.exceptions.CellNotFound as e:
                print "Pair not exist"
                # Create new columns
                ws.add_cols(3)
                i = ws.col_count - 2
                
                # Write new header for new pair
                ws.update_cell(1, i, key)
                ws.update_cell(2, i, "Price")
                ws.update_cell(2, i+1, "Volume")
                ws.update_cell(2, i+2, "Change (%)")
                
                # Write data of pair
                ws.update_cell(count, i, value["last_price"])
                ws.update_cell(count, i+1, value["volume"])
                ws.update_cell(count, i+2, value["change"])
            
    except gspread.exceptions.WorksheetNotFound as e:
        print type(e)
        print "chua co sheet " + k
        # Create sheet 
        ws = sh.add_worksheet(k, 33, v["column"])
        print "da tao sheet " + k
        
        # Create sheet header
        i = 2
        ws.update_cell(2, 1, "Date")
        ws.update_cell(3, 1, v["data"]["time"])
        for key, value in v["data"]["pair"].iteritems():
            ws.update_cell(1, i, key)
            ws.update_cell(2, i, "Price")
            ws.update_cell(2, i+1, "Volume")
            ws.update_cell(2, i+2, "Change (%)") 
            
            # Write the first record
            ws.update_cell(3, i, value["last_price"])
            ws.update_cell(3, i+1, value["volume"])
            ws.update_cell(3, i+2, value["change"])
            
            i = i + 3
          

da co sheet 2018-01-binance
4
da co sheet 2018-01-bittrex
4
